## Naive RAG : QA Pipeline 

In [1]:
#Init Libraries

#Fitting Libraries
from components import Components
from llama_index.core import Settings

#Additional Core Libraries
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core import PromptTemplate,Document 
from llama_index.core.node_parser import SentenceSplitter

import pandas as pd

e:\Github Repositories\RAG-FYP2024\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#initialise components, embedder and retriever
rag_components = Components("Snowflake/snowflake-arctic-embed-l","","llama3-8b-8192")

In [3]:
Settings.embed_model = rag_components.get_embedding_model()
Settings.llm = rag_components.get_groq_llm()

Embedding model loaded!


In [4]:
vector_store = ChromaVectorStore(chroma_collection=rag_components.get_db())
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#### Loading Context

In [5]:
df = pd.read_csv('../data/MainDataset/data/Context.csv')

In [6]:
context_list = []
for contexts in df['context']:
    context_list.append(Document(text=contexts))

In [7]:
pipeline = IngestionPipeline(
    transformations=[
          SentenceSplitter(chunk_size=256, chunk_overlap=100),
    ],
    vector_store=vector_store,
)

In [8]:
documents = pipeline.run(documents=context_list)

In [9]:
index = VectorStoreIndex(documents, storage_context=storage_context,similarity_top_k=5) 

In [10]:
query_engine = index.as_query_engine(streaming=True,similarity_top_k=3)

In [18]:
qa_prompt_template_str = """
Context: {context_str}
Instructions:
- Be helpful and answer questions concisely. If you don't know the answer, say 'I don't know'
- Utilize the context provided for accurate and specific information.
- Keep your answers short and straight to the point
Question: {query_str}
"""


qa_prompt_template = PromptTemplate(qa_prompt_template_str)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template":qa_prompt_template}
)


In [12]:
query_engine.update_prompts(
    {"response_synthesizer:refine_template":PromptTemplate("")}
)


### Loading Questions

In [19]:
df_qa = pd.read_csv('../data/MainDataset/data/QA.csv')

In [20]:
question_list = []
for questions in df_qa['question']:
    question_list.append(questions)
    

In [21]:
import time
for i in range(len(question_list)): 
    time.sleep(3)
    print(question_list[i])
    response = query_engine.query(question_list[i])
    response.print_response_stream()
    rowIndex = df_qa.index[i]

    df_qa.at[rowIndex, 'gen_answer'] = response.response_txt
    print('')
    print("--"*100)
    print(f'{i}/{128}')

what was the climate like in the cretaceous period
The Cretaceous period was a warmer time, with no ice at the poles. The world was generally warmer, with tropical plants and ferns found in fossils. The poles were cooler than the lower latitudes, but overall, the climate was warmer than today's.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0/128
who first used a touch screen in a smartphone
According to the context, Sam Hurst and Elographics developed the first true touch screen incorporating a transparent surface in 1974.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
1/128
how do the digestive system and endocrine system work together
The digestive and endocrine systems work toge

In [22]:
df_qa.head()

,question,answers,gen_answer
0,what was the climate like in the cretaceous pe...,Warmer,"The Cretaceous period was a warmer time, with ..."
1,who first used a touch screen in a smartphone,IBM Simon,"According to the context, Sam Hurst and Elogra..."
2,how do the digestive system and endocrine syst...,The digestive and endocrine systems work toget...,The digestive and endocrine systems work toget...
3,what does leukemia do to the body,It harms the body's ability to make healthy bl...,Leukemia harms the body's ability to make heal...
4,meaning of the word attitude,It is a way of feeling or acting toward a pers...,"According to the context, the meaning of the w..."


In [23]:
df_qa.to_csv(f'../data/MainDataset/results/Official/{rag_components.model_name}_Naive_RAG.csv',index=False)